In [1]:
using Revise,DelimitedFiles,Random,StaticArrays
using ParallelTemperingMonteCarlo,StructArrays
using BenchmarkTools
script_folder = @__DIR__ # folder where this script is located
data_path = joinpath(script_folder, "data") 

"/home/grayseff/Code/ParallelTemperingMonteCarlo.jl/scripts/data"

Ensemble, potential, params etc

In [2]:
n_atoms = 38

ti = 300.
tf = 1300.
n_traj = 28

temp = TempGrid{n_traj}(ti,tf) 

mc_cycles = 10000
mc_sample = 1

displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

evtohartree = 0.0367493
nmtobohr = 18.8973

#parameters taken from L Vocadlo etal J Chem Phys V120N6 2004
n = 8.482
m = 4.692
ϵ = evtohartree*0.0370
a = 0.25*nmtobohr
C = 27.561

pot = EmbeddedAtomPotential(n,m,ϵ,C,a)
ensemble = NVT(n_atoms)
move_strat = MoveStrategy(ensemble)



pos_cu38 = [[2.3603476948363165, 2.3603476948363165, 0.0],
[6.994369407022418, 2.33998871029911, 0.0],
[2.33998871029911, 6.994369407022418, 0.0],
[-2.3603476948363165, 2.3603476948363165, 0.0],
[-6.994369407022418, 2.33998871029911, 0.0],
[-2.33998871029911, 6.994369407022418, 0.0],
[-2.3603476948363165, -2.3603476948363165, 0.0],
[-6.994369407022418, -2.33998871029911, 0.0],
[-2.33998871029911, -6.994369407022418, 0.0],
[2.3603476948363165, -2.3603476948363165, 0.0],
[6.994369407022418, -2.33998871029911, 0.0],
[2.33998871029911, -6.994369407022418, 0.0],
[0.0, 0.0, 3.3380357219419614],
[4.84532317769689, 0.0, 3.4261608756649893],
[-4.84532317769689, 0.0, 3.4261608756649893],
[0.0, 4.84532317769689, 3.4261608756649893],
[0.0, -4.84532317769689, 3.4261608756649893],
[4.667179058660764, 4.667179058660764, 3.2911441531516483],
[-4.667179058660764, 4.667179058660764, 3.2911441531516483],
[-4.667179058660764, -4.667179058660764, 3.2911441531516483],
[4.667179058660764, -4.667179058660764, 3.2911441531516483],
[0.0, 0.0, -3.3380357219419614],
[4.84532317769689, 0.0, -3.4261608756649893],
[-4.84532317769689, 0.0, -3.4261608756649893],
[0.0, 4.84532317769689, -3.4261608756649893],
[0.0, -4.84532317769689, -3.4261608756649893],
[4.667179058660764, 4.667179058660764, -3.2911441531516483],
[-4.667179058660764, 4.667179058660764, -3.2911441531516483],
[-4.667179058660764, -4.667179058660764, -3.2911441531516483],
[4.667179058660764, -4.667179058660764, -3.2911441531516483], 
[2.327190348361654, 2.327190348361654, 6.600387922922003],
[-2.327190348361654, 2.327190348361654, 6.600387922922003],
[-2.327190348361654, -2.327190348361654, 6.600387922922003],
[2.327190348361654, -2.327190348361654, 6.600387922922003],
[2.327190348361654, 2.327190348361654, -6.600387922922003],
[-2.327190348361654, 2.327190348361654, -6.600387922922003],
[-2.327190348361654, -2.327190348361654, -6.600387922922003],
[2.327190348361654, -2.327190348361654, -6.600387922922003]]
shuffle!(pos_cu38)
n_bin = 100
AtoBohr = 1.8897259886

bc_cu38 = SphericalBC(radius=14*AtoBohr) 
start_config = Config(pos_cu38, bc_cu38)


Config{38, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[-4.667179058660764, -4.667179058660764, 3.2911441531516483], [2.327190348361654, -2.327190348361654, 6.600387922922003], [-4.667179058660764, 4.667179058660764, 3.2911441531516483], [2.33998871029911, 6.994369407022418, 0.0], [-2.327190348361654, -2.327190348361654, 6.600387922922003], [0.0, 4.84532317769689, 3.4261608756649893], [-6.994369407022418, -2.33998871029911, 0.0], [-2.327190348361654, 2.327190348361654, -6.600387922922003], [4.667179058660764, 4.667179058660764, -3.2911441531516483], [-4.667179058660764, 4.667179058660764, -3.2911441531516483]  …  [-2.327190348361654, 2.327190348361654, 6.600387922922003], [-6.994369407022418, 2.33998871029911, 0.0], [2.3603476948363165, -2.3603476948363165, 0.0], [2.33998871029911, -6.994369407022418, 0.0], [-2.327190348361654, -2.327190348361654, -6.600387922922003], [2.327190348361654, 2.327190348361654, -6.600387922922003], [6.994369407022418, -2.33998871029911, 0.0], [2.3271

RunnerPotential

We require a different view for our calculation of tpz and g_offset

In [3]:
struct RadialType2a{T} <: RadialSymmFunction{T}
    eta::T
    r_cut::T
    type_vec::Int
    G_offset::SVector{4,T}
    G_norm::SVector{4,T}
end
function RadialType2a{T}(eta,r_cut,type_vector) where {T}
    return RadialType2a(eta,r_cut,type_vector,SVector{2}(0.,0.),SVector{2}(1.,1.))
end
function RadialType2a{T}(eta,r_cut,type_vector,G_vals_cu::Vector,G_vals_zn) where {T}
    G_norm1 = 1/(G_vals_cu[1][1] - G_vals_cu[1][2])
    G_offset1 = -G_vals_cu[1][2]*G_norm1

    G_norm2 = 1/(G_vals_cu[2][1] - G_vals_cu[2][2])
    G_offset2 = -G_vals_cu[2][2]*G_norm2

    G_norm3 = 1/(G_vals_zn[1][1] - G_vals_zn[1][2])
    G_offset3 = -G_vals_zn[1][2]*G_norm3

    G_norm4 = 1/(G_vals_zn[2][1] - G_vals_zn[2][2])
    G_offset4 = -G_vals_zn[2][2]*G_norm4


    return RadialType2a(eta,r_cut,type_vector,SVector{4}(G_offset1,G_offset2,G_offset3,G_offset4),SVector{4}(G_norm1,G_norm2,G_norm3,G_norm4))

end
struct AngularType3a{T} <:AngularSymmFunction{T}
    eta::T
    lambda::T
    zeta::T
    r_cut::T
    type_vec::Int
    tpz::SVector{6,T}
    G_offset::SVector{6,T}
    #G_norm::T
end
function AngularType3a{T}(eta,lambda,zeta,r_cut,type_vec) where {T}
    tpz = 2.0^(1-zeta)

    return AngularType3a(eta,lambda,zeta,r_cut,type_vec,tpz,SVector{6}(0.,0.,0.,0.,0.,0.))
end
function AngularType3a{T}(eta,lambda,zeta,r_cut,type_vector,G_valsa::Vector,G_valsb::Vector) where {T}

    G_norm1 = 1/(G_valsa[1][1] - G_valsa[1][2])
    G_offset1 = -G_valsa[1][2]*G_norm1
    tpz1 = 2.0^(1-zeta)*G_norm1

    G_norm2 = 1/(G_valsa[2][1] - G_valsa[2][2])
    G_offset2 = -G_valsa[2][2]*G_norm2
    tpz2 = 2.0^(1-zeta)*G_norm2

    G_norm3 = 1/(G_valsa[3][1] - G_valsa[3][2])
    G_offset3 = -G_valsa[3][2]*G_norm3
    tpz3 = 2.0^(1-zeta)*G_norm3

    G_norm4 = 1/(G_valsb[1][1] - G_valsb[1][2])
    G_offset4 = -G_valsb[1][2]*G_norm4
    tpz4 = 2.0^(1-zeta)*G_norm4

    G_norm5 = 1/(G_valsb[2][1] - G_valsb[2][2])
    G_offset5 = -G_valsb[2][2]*G_norm5
    tpz5 = 2.0^(1-zeta)*G_norm5

    G_norm6 = 1/(G_valsb[3][1] - G_valsb[3][2])
    G_offset6 = -G_valsb[3][2]*G_norm6
    tpz6 = 2.0^(1-zeta)*G_norm6

    return AngularType3a(eta,lambda,zeta,r_cut,type_vector,SVector{6}(tpz1,tpz2,tpz3,tpz4,tpz5,tpz6),SVector{6}(G_offset1,G_offset2,G_offset3,G_offset4,G_offset5,G_offset6))
end


In [4]:
X = [ 11              0.001   0.000  11.338
 10              0.001   0.000  11.338
 11              0.020   0.000  11.338
 10              0.020   0.000  11.338
 11              0.035   0.000  11.338
 10              0.035   0.000  11.338
 11              0.100   0.000  11.338
 10              0.100   0.000  11.338
 11              0.400   0.000  11.338
 10              0.400   0.000  11.338]
radsymmvec = []
V = [[0.0001,1,1,11.338],[0.0001,-1,2,11.338],[0.003,-1,1,11.338],[0.003,-1,2,11.338],[0.008,-1,1,11.338],[0.008,-1,2,11.338],[0.008,1,2,11.338],[0.015,1,1,11.338],[0.015,-1,2,11.338],[0.015,-1,4,11.338],[0.015,-1,16,11.338],[0.025,-1,1,11.338],[0.025,1,1,11.338],[0.025,1,2,11.338],[0.025,-1,4,11.338],[0.025,-1,16,11.338],[0.025,1,16,11.338],[0.045,1,1,11.338],[0.045,-1,2,11.338],[0.045,-1,4,11.338],[0.045,1,4,11.338],[0.045,1,16,11.338],[0.08,1,1,11.338],[0.08,-1,2,11.338],[0.08,-1,4,11.338],[0.08,1,4,11.338]]
T = [111,110,100]
angularsymmvec = []
#-------------------------------------------#
#-----------Including scaling data----------#
#-------------------------------------------#
file = open(joinpath(data_path,"scaling.data")) # full path "./data/scaling.data"
scalingvalues = readdlm(file)
close(file)
G_value_vec = []
for row in eachrow(scalingvalues[1:88,:])
    max_min = [row[4],row[3]]
    push!(G_value_vec,max_min)
end
for symmindex in eachindex(eachrow(X))
    row = X[symmindex,:]
    radsymm = RadialType2{Float64}(row[2],row[4],Int(row[1]),G_value_vec[symmindex])
    push!(radsymmvec,radsymm)
end
let n_index = 10
for element in V
    for types in T

        n_index += 1

        symmfunc = AngularType3{Float64}(element[1],element[2],element[3],11.338,types,G_value_vec[n_index])

        push!(angularsymmvec,symmfunc)
    end
end
end

totalsymmvec = vcat(radsymmvec,angularsymmvec)
num_nodes::Vector{Int32} = [88, 20, 20, 1]
activation_functions::Vector{Int32} = [1, 2, 2, 1]
file = open(joinpath(data_path, "weights.029.data"), "r+") # "./data/weights.029.data"
weights=readdlm(file)
close(file)
weights = vec(weights)
nnp = NeuralNetworkPotential(num_nodes,activation_functions,weights)
testpotential = RuNNerPotential(nnp,radsymmvec,angularsymmvec)
statest,stratest,resultest,nsteptest,startcounttest=initialisation(mc_params,temp,start_config,testpotential,ensemble)

(MCState{Float64, 38, SphericalBC{Float64}, NNPVariables{Float64}, NVTVariables{Float64}}[MCState{Float64, 38, SphericalBC{Float64}, NNPVariables{Float64}, NVTVariables{Float64}}(300.0, 1052.5832842103239, Config{38, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[-4.667179058660764, -4.667179058660764, 3.2911441531516483], [2.327190348361654, -2.327190348361654, 6.600387922922003], [-4.667179058660764, 4.667179058660764, 3.2911441531516483], [2.33998871029911, 6.994369407022418, 0.0], [-2.327190348361654, -2.327190348361654, 6.600387922922003], [0.0, 4.84532317769689, 3.4261608756649893], [-6.994369407022418, -2.33998871029911, 0.0], [-2.327190348361654, 2.327190348361654, -6.600387922922003], [4.667179058660764, 4.667179058660764, -3.2911441531516483], [-4.667179058660764, 4.667179058660764, -3.2911441531516483]  …  [-2.327190348361654, 2.327190348361654, 6.600387922922003], [-6.994369407022418, 2.33998871029911, 0.0], [2.3603476948363165, -2.3603476948363165, 0.0], [2.3399887102

In [5]:
X = [ 2              0.001   0.000  11.338
 #10              0.001   0.000  11.338
 2              0.020   0.000  11.338
 #10              0.020   0.000  11.338
 2              0.035   0.000  11.338
 #10              0.035   0.000  11.338
 2              0.100   0.000  11.338
 #10              0.100   0.000  11.338
 2              0.400   0.000  11.338
 #10              0.400   0.000  11.338
 ]

radsymmvec = []
#--------------------------------------------#
#--------Vector of angular symm values-------#
#--------------------------------------------#
V = [[0.0001,1,1,11.338],
[0.0001,-1,2,11.338],
[0.003,-1,1,11.338],
[0.003,-1,2,11.338],
[0.008,-1,1,11.338],
[0.008,-1,2,11.338],
[0.008,1,2,11.338],
[0.015,1,1,11.338],
[0.015,-1,2,11.338],
[0.015,-1,4,11.338],
[0.015,-1,16,11.338],
[0.025,-1,1,11.338],
[0.025,1,1,11.338],
[0.025,1,2,11.338],
[0.025,-1,4,11.338],
[0.025,-1,16,11.338],
[0.025,1,16,11.338],
[0.045,1,1,11.338],
[0.045,-1,2,11.338],
[0.045,-1,4,11.338],
[0.045,1,4,11.338],
[0.045,1,16,11.338],
[0.08,1,1,11.338],
[0.08,-1,2,11.338],
[0.08,-1,4,11.338],
[0.08,1,4,11.338]]
T = [111,110,100]
angularsymmvec = []
#-------------------------------------------#
#-----------Including scaling data----------#
#-------------------------------------------#
file = open(joinpath(data_path,"scaling.data")) # full path "./data/scaling.data"
scalingvalues = readdlm(file)
close(file)
G_value_vec = []
for row in eachrow(scalingvalues[1:88,:])
    max_min = [row[4],row[3]]
    push!(G_value_vec,max_min)
end
G_value_vec_b = []
for row in eachrow(scalingvalues[89:end,:])
    max_min = [row[4],row[3]]
    push!(G_value_vec_b,max_min)
end
for symmindex in eachindex(eachrow(X))
    row = X[symmindex,:]
    radsymm = RadialType2a{Float64}(row[2],row[4],Int(row[1]),[G_value_vec[(symmindex-1)*2 + 1],G_value_vec[(symmindex-1)*2 + 2]],[G_value_vec_b[(symmindex-1)*2 + 1],G_value_vec_b[(symmindex-1)*2 + 2]])
    push!(radsymmvec,radsymm)
end
let n_index = 10
let j_index = 0
for element in V
    #for types in T

        j_index += 1

        symmfunc = AngularType3a{Float64}(element[1],element[2],element[3],11.338,2,[G_value_vec[n_index + (j_index-1)*3 + 1],G_value_vec[n_index + (j_index-1)*3 + 2],G_value_vec[n_index + (j_index-1)*3 + 3] ],[G_value_vec_b[n_index + (j_index-1)*3 + 1],G_value_vec_b[n_index + (j_index-1)*3 + 2],G_value_vec_b[n_index + (j_index-1)*3 + 3] ])

        push!(angularsymmvec,symmfunc)
    #end
end
end
end
#---------------------------------------------------#
#------concatenating radial and angular values------#
#---------------------------------------------------#
totalsymmvec = vcat(radsymmvec,angularsymmvec)
#--------------------------------------------------#
#-----------Initialising the nnp weights-----------#
#--------------------------------------------------#
num_nodes::Vector{Int32} = [88, 20, 20, 1]
activation_functions::Vector{Int32} = [1, 2, 2, 1]


4-element Vector{Int64}:
 1
 2
 2
 1

In [6]:

#runnerpotential = RuNNerPotential(nnp,radsymmvec,angularsymmvec)
struct RuNNerPotential2Atom{Nrad,Nang,N1,N2} <: AbstractMachineLearningPotential
    nnp1::NeuralNetworkPotential
    nnp2::NeuralNetworkPotential
    radsymfunctions::StructVector{RadialType2a{Float64}}
    angsymfunctions::StructVector{AngularType3a{Float64}}
    r_cut::Float64 
#    g_offsets::SVector{Nrad*2+Nang*3,Float64}
#    tpz::SVector{Nrad*2+Nang*3,Float64}
end
function RuNNerPotential2Atom(nnp1,nnp2,radsymvec,angsymvec,n1,n2)#,g_offsets_vec)
    r_cut = radsymvec[1].r_cut
    nrad = length(radsymvec)
    nang = length(angsymvec)
    radvec=StructVector([rsymm for rsymm in radsymvec])
    angvec = StructVector([asymm for asymm in angsymvec])

    # g_norm_vec = 1 ./ (g_offsets_vec[1] .- g_offsets_vec[2])
    # g_offsets = -1*( g_offsets_vec[2] .* g_norm_vec )
    #tpzvec = zeros(nrad*2+nang*3)
    # for idx in 1:nang
    #    truidx=nrad*2 + (idx - 1)*3
    #    tpzval = 2^( 1 - angvec[idx].zeta )
    #    tpz[truidx+1] = tpzval*g_norm_vec[truidx+1]
    #    tpz[truidx+2] = tpzval*g_norm_vec[truidx+2]
    #    tpz[truidx+3] = tpzval*g_norm_vec[truidx+3]
    # end
    return RuNNerPotential2Atom{nrad,nang,n1,n2}(nnp1,nnp2,radvec,angvec,r_cut)#,SVector{nrad*2+nang*3}(g_offsets),SVector{nrad*2+nang*3}(tpz) )
end


RuNNerPotential2Atom

In [32]:
file = open(joinpath(data_path, "weights.029.data"), "r+") # "./data/weights.029.data"
weights=readdlm(file)
close(file)
weights = vec(weights)
nnpcu = NeuralNetworkPotential(num_nodes,activation_functions,weights)

file2=open(joinpath(data_path, "weights.030.data"),"r+") #./data/weights.030.data
weights2=readdlm(file2)
close(file2)
weights2 = vec(weights2)
nnpzn= NeuralNetworkPotential(num_nodes,activation_functions,weights2)


runnerpotential = RuNNerPotential2Atom(nnpcu,nnpzn,radsymmvec,angularsymmvec,32,6)

RuNNerPotential2Atom{5, 26, 32, 6}(NeuralNetworkPotential(4, 2221, Int32[88, 20, 20, 1], Int32[1, 2, 2, 1], [-1.0768943782, 0.3563458393, -1.3084861447, 0.1127640916, -0.5646591931, -0.4969830793, 1.423770514, -0.2005662393, -0.8957859374, 0.6076110858  …  0.1939524848, -0.0342903191, -0.0231042009, 0.0718481968, -0.0828320122, -0.0728501885, 0.0306628826, 0.0486053813, -0.0867018862, 0.0441651841]), NeuralNetworkPotential(4, 2221, Int32[88, 20, 20, 1], Int32[1, 2, 2, 1], [-0.2990596476, 0.50210302, -0.0769338744, -1.1006272074, 0.2955859359, 1.7429579447, -1.6013368538, 0.2350663304, 0.5352038521, 0.0051334333  …  -0.07023029, -0.0274223406, 0.0379881304, 0.1032826216, -0.0671415566, 0.0873441204, 0.0024981831, -0.1005392202, 0.011513595, -0.0270711056]), RadialType2a{Float64}[RadialType2a{Float64}(0.001, 11.338, 2, [-0.029892621990594427, -0.0, -0.0, -0.0], [0.06634475788973929, 0.09828103647521905, 0.07273034544948624, 0.07540689868304344]), RadialType2a{Float64}(0.02, 11.338, 2, [-

In [8]:
testvector = rand(2,8)
test2vector = testvector[2:-1:1,:]

2×8 Matrix{Float64}:
 0.0660521  0.0990409  0.660522  0.42194   …  0.919693  0.205028  0.261873
 0.709831   0.295217   0.353154  0.214984     0.749844  0.802476  0.26784

In [9]:
theta_part(θ,λ,ζ) = (1+λ*θ)^ζ
exponential_part(η,r2_ij,r2_ik,r2_jk,f_ij,f_ik,f_jk) = exp(-η*(r2_ij+r2_ik+r2_jk))* f_ij * f_ik * f_jk
exponential_part(η,rsum,f_prod) = exp(-η*(rsum))*f_prod
"""
    symmfunc_calc(θ_vec,r2_ij,r2_ik,r2_jk,f_ij,f_ik,f_jk,η,λ,ζ)
Calculates the three g_values corresponding to the three atoms iterated over, builds the foundation of the total symm function as calculated below.
"""
function symmfunc_calc(θ_vec,r2_ij,r2_ik,r2_jk,f_ij,f_ik,f_jk,η,λ,ζ)

    exp_part = exponential_part(η,r2_ij,r2_ik,r2_jk,f_ij,f_ik,f_jk)
    g_values = MVector{3}([exp_part* theta_part(θ,λ,ζ) for θ in θ_vec])
    
    return g_values
end

calc_one_symm_val(r2_ij,fc_ij,η) = ifelse(fc_ij!=0. && fc_ij!=1., fc_ij*exp(-η*r2_ij), 0.)

function calc_one_symm_val(position1,position2,position3,r2_ij,r2_ik,r2_jk,f_ij,f_ik,f_jk,η,λ,ζ)
    θ_vec = all_angular_measure(position1,position2,position3,r2_ij,r2_ik,r2_jk)
    
    g_vals = symmfunc_calc(θ_vec,r2_ij,r2_ik,r2_jk,f_ij,f_ik,f_jk,η,λ,ζ)

    return g_vals
end

calc_one_symm_val (generic function with 2 methods)

In [145]:
function calc_symm_vals!(position,dist2_mat,f_mat,g_vec,n1,n2,symm_func::RadialType2a)
    N = n1+n2
    g_norm , η = symm_func.G_norm , symm_func.eta
    # CuCu populates row 1 for both i and j
    for atomindex in 1:n1
        for index2 in atomindex+1:n1

            g_val =  calc_one_symm_val(dist2_mat[atomindex,index2],f_mat[atomindex,index2],η)

            g_vec[1,atomindex] += g_val 
            g_vec[1,index2] += g_val
        end
    end
    #ZnZn populates row 2 for both i and j
    for atomindex in n1+1:N
        for index2 in atomindex+1:N

            g_val =  calc_one_symm_val(dist2_mat[atomindex,index2],f_mat[atomindex,index2],η)

            g_vec[2,atomindex] += g_val 
            g_vec[2,index2] += g_val
        end
    end
    #CuZn is row 2 for i row 1 for Zn
    for atomindex in 1:n1
        for index2 in n1+1:N

            g_val =  calc_one_symm_val(dist2_mat[atomindex,index2],f_mat[atomindex,index2],η)

            g_vec[2,atomindex] += g_val 
            g_vec[1,index2] += g_val
        end
    end
    
    g_vec[1,1:n1] = g_vec[1,1:n1] .* g_norm[1] .+ symm_func.G_offset[1]
    g_vec[2,1:n1] = g_vec[2,1:n1] .* g_norm[2] .+ symm_func.G_offset[2]

    g_vec[1,1+n1:N] = g_vec[1,1+n1:N] .* g_norm[3] .+ symm_func.G_offset[3]
    g_vec[2,1+n1:N] = g_vec[2,1+n1:N] .* g_norm[4] .+ symm_func.G_offset[4]

    return g_vec
end

function calc_symm_vals!(positions,dist2_mat,f_mat,g_vec,n1,n2,symm_func::AngularType3a)
    N= n1+n2
    η,λ,ζ,tpz = symm_func.eta,symm_func.lambda,symm_func.zeta,symm_func.tpz
 #   CuCuCu row 1
    for atomindex in 1:n1
        for index2 in atomindex+1:n1
            for index3 in index2+1:n1
                g_vals=calc_one_symm_val(positions[atomindex],positions[index2],positions[index3],dist2_mat[atomindex,index2],dist2_mat[atomindex,index3],dist2_mat[index2,index3],f_mat[atomindex,index2],f_mat[atomindex,index3],f_mat[index2,index3],η,λ,ζ)

                g_vec[1,atomindex] += g_vals[1]
                g_vec[1,index2] += g_vals[2]
                g_vec[1,index3] += g_vals[3]
            end
        end
    end
#   ZnZnZn row 3
    for atomindex in n1+1:N
        for index2 in atomindex+1:N
            for index3 in index2+1:N
                g_vals=calc_one_symm_val(positions[atomindex],positions[index2],positions[index3],dist2_mat[atomindex,index2],dist2_mat[atomindex,index3],dist2_mat[index2,index3],f_mat[atomindex,index2],f_mat[atomindex,index3],f_mat[index2,index3],η,λ,ζ)

                g_vec[3,atomindex] += g_vals[1]
                g_vec[3,index2] += g_vals[2]
                g_vec[3,index3] += g_vals[3]
            end
        end
    end
#CuCuZn row 2 for Cu, row 1 for Zn
    for atomindex in 1:n1
        for index2 in atomindex+1:n1
            for index3 in n1+1:N
                
                g_vals=calc_one_symm_val(positions[atomindex],positions[index2],positions[index3],dist2_mat[atomindex,index2],dist2_mat[atomindex,index3],dist2_mat[index2,index3],f_mat[atomindex,index2],f_mat[atomindex,index3],f_mat[index2,index3],η,λ,ζ)
                

                g_vec[2,atomindex] += g_vals[1]
                g_vec[2,index2] += g_vals[2]
                g_vec[1,index3] += g_vals[3]
            end
        end
    end
# CuZnZn row 3 for Cu row 2 for Zn
    for atomindex in 1:n1
        for index2 in n1+1:N
            for index3 in index2+1:N
                g_vals=calc_one_symm_val(positions[atomindex],positions[index2],positions[index3],dist2_mat[atomindex,index2],dist2_mat[atomindex,index3],dist2_mat[index2,index3],f_mat[atomindex,index2],f_mat[atomindex,index3],f_mat[index2,index3],η,λ,ζ)

                g_vec[3,atomindex] += g_vals[1]
                g_vec[2,index2] += g_vals[2]
                g_vec[2,index3] += g_vals[3]
            end
        end
    end
    g_vec[1,1:n1] = g_vec[1,1:n1].*tpz[1] .+ symm_func.G_offset[1]
    g_vec[2,1:n1] = g_vec[2,1:n1].*tpz[2] .+ symm_func.G_offset[2]
    g_vec[3,1:n1] = g_vec[3,1:n1].*tpz[3] .+ symm_func.G_offset[3]
    
    g_vec[1,n1+1:N] = g_vec[1,n1+1:N].*tpz[4] .+ symm_func.G_offset[4]
    g_vec[2,n1+1:N] = g_vec[2,n1+1:N].*tpz[5] .+ symm_func.G_offset[5]
    g_vec[3,n1+1:N] = g_vec[3,n1+1:N].*tpz[6] .+ symm_func.G_offset[6]

    return g_vec
end

calc_symm_vals! (generic function with 2 methods)

These would calculate the complete matrices, of length 2 or 3 (for radial vs angular) for a single symmetry value. That is, for our particular example, we go through each symmetry function and populate a combinatorically sound G matrix

In [146]:
function total_symm_calc(positions,dist2_mat,f_mat,radsymmfunctions,angsymmfunctions,nrad,nang,n1,n2)
    g_mat = zeros(nrad*2 + nang*3 , length(positions))

    for g_index in 1:nrad
        idx=(g_index-1)*2+1
        g_mat[idx:idx+1,:] = calc_symm_vals!(positions,dist2_mat,f_mat,g_mat[idx:idx+1,:],n1,n2,radsymmfunctions[g_index])
    end
    for g_index in 1:nang 
        idx = nrad*2 + (g_index-1)*3 + 1 

        g_mat[idx:idx+2,:] = calc_symm_vals!(positions,dist2_mat,f_mat,g_mat[idx:idx+2,:],n1,n2,angsymmfunctions[g_index])
    end
    return g_mat
end

total_symm_calc (generic function with 2 methods)

In [147]:
dis2mat=get_distance2_mat(start_config)
fmat = cutoff_function.(sqrt.(dis2mat),Ref(runnerpotential.r_cut))
#g_vec_test = zeros(3,38)
positions = start_config.pos
testsymmfunc = runnerpotential.angsymfunctions[1]
g_vec_test = zeros(3,38)
#X = calc_symm_vals!(positions,dis2mat,fmat,g_vec_test,30,8,runnerpotential.angsymfunctions[1])

g = calc_symm_vals!(positions,dis2mat,fmat,g_vec_test,30,8,testsymmfunc)

3×38 Matrix{Float64}:
 0.215266    0.115751  0.2312       …  0.232456    0.495094   0.249096
 0.0800042   0.261549  0.0454572       0.155545    0.508018   0.387751
 0.00386648  0.059058  0.000287002     0.00752146  0.0248618  0.0269029

In [148]:
gmat=total_symm_calc(start_config.pos,dis2mat,fmat,runnerpotential.radsymfunctions,runnerpotential.angsymfunctions,length(runnerpotential.radsymfunctions),length(runnerpotential.angsymfunctions),32,6)


88×38 Matrix{Float64}:
 0.369506     0.281048   0.359579     0.329095     …  0.71955     0.491495
 0.010325     0.141364   0.0250313    0.0701882       0.12057     0.106197
 0.41575      0.301073   0.409381     0.391113        0.766125    0.519433
 0.00325322   0.163532   0.0121555    0.0376877       0.144834    0.134867
 0.434908     0.307028   0.431018     0.420083        0.765621    0.520055
 0.00124181   0.175765   0.006551     0.0214746    …  0.161052    0.15313
 0.418896     0.282958   0.418639     0.417932        0.664259    0.461725
 1.71073e-5   0.200526   0.000397098  0.00144         0.204062    0.194975
 0.0730804    0.0489541  0.0730804    0.0730804       0.113693    0.0819406
 8.38881e-15  0.0386131  1.53215e-10  5.24035e-10     0.0534105   0.045916
 ⋮                                                 ⋱              
 0.149938     0.072194   0.149911     0.149812        0.284402    0.150661
 6.71259e-7   0.108409   4.75855e-5   0.000218868  …  0.279725    0.203601
 2.21847e-

And then, we move on to calculation of the delta-matrix. In this case, we want to calculate a unique triple, retrieve the three individual delta-values and then inputting them into the correct vector in the same fashion as above. 

In [14]:
function new_radial_symm_val!(rnew_ij,r2_ij,fnew_ij,f2_ij,η,g_norm)
    return exponential_part(η,r_newij,fnew_ij*g_norm) - exponential_part(η,r2_ij,f2_ij*g_norm)
end
function calc_new_symmetry_value!(g_vector,atomindex,dist2_mat,new_dist2_vector,f_matrix,new_f_vector,n1,n2,symmetry_function::RadialType2)
    η,g_norm = symmetry_function.eta,symmetry_function.G_norm
    N = n1+n2
    g_vector[1,atomindex] = 0.
    g_vector[2,atomindex] = 0.
    #calculate 11
    for index2 in 1:n1
        if index2 != atomindex
            g_new = new_radial_symm_val!(new_dist2_vector[index2],dist2_mat[atomindex,index2],new_f_vector[index2],f_matrix[atomindex,index2],η,g_norm)

            g_vector[1,atomindex] += g_new
            g_vector[1,index2] = g_new
        end
    end
    #calculate 10
    for index2 in n1+1:N
        if index2 != atomindex
            g_new = new_radial_symm_val!(new_dist2_vector[index2],dist2_mat[atomindex,index2],new_f_vector[index2],f_matrix[atomindex,index2],η,g_norm)

            g_vector[2,atomindex] += g_new
            g_vector[2,index2] = g_new
        end
    end
    #invert if moved atom is type 0 
    if atomindex > n1
        g_vector = g_vector[2:-1:1,:]
    end

    return g_vector

end

calc_new_symmetry_value! (generic function with 1 method)

Lastly we have to look at calculating a new angular symmetry value. This should return three totally nonsymmetric values which are then assigned to their respective indices in the list. We will typically retrieve a matrix of three rows, the top row should be '111' the next '110' and the last '100'. If we calculate assuming automatically that our atom is of type 1, we calculate the double n1 loop and put it in the top row, the double n2 loop in the three row and the n1n2 loop in the middle. We can then invert the matrix entirely if we actually have atom of type 0.



In [15]:
function new_angular_symm_vals(newposition,position1,position2,position3,rnew_ij,rnew_ik,r2_ij,r2_ik,r2_jk,fnew_ij,fnew_ik,f_ij,f_ik,f_jk,η,λ,ζ,tpz)

    θ_new,θ_old = all_angular_measure(newposition,position2,position3,rnew_ij,rnew_ik,r2_jk),all_angular_measure(position1,position2,position3,r2_ij,r2_ik,r2_jk)

    θ_val_old,θ_val_new = theta_part(θ_old,λ,ζ),theta_part(θ_new,λ,ζ)

    exp_new,exp_old = exponential_part(η,rnew_ij,rnew_ik,r2_jk,fnew_ij,fnew_ik,f_jk),exponential_part(η,r2_ij,r2_ik,r2_jk,f_ij,f_ik,f_jk)

    return tpz*(θ_val_new.*exp_new .- θ_val_old.*exp_old)
end

function calc_new_symmetry_value!(g_vector,positions,newposition,atomindex,dist2_mat,new_dist2_vector,f_matrix,new_f_vector,n1,n2,symmetry_function::AngularType3)

    N = n1+n2 

    η,λ,ζ,tpz = symmetry_function.eta,symmetry_function.lambda,symmetry_function.zeta,symmetry_function.tpz

    temp_vector = zeros(4,N)
    i_val = ifelse(atomindex <= n1 , [1,2,3] , [2,3,1])
    #calculate 111
    for j_index in 1:n1
        if j_index != atomindex
            for k_index in j_index+1:n1
                if k_index != atomindex

                    g_vals = new_angular_symm_vals(newposition,positions[atomindex],positions[j_index],positions[k_index],new_dist2_vector[indexj],new_dist2_vector[indexk],dist2_mat[indexi,indexj],dist2_mat[indexi,indexk],dist2_mat[indexj,indexk],new_f_vector[indexj],new_f_vector[indexk],f_matrix[indexi,indexj],f_matrix[indexi,indexk],f_matrix[indexj,indexk],η,λ,ζ,tpz)

                    temp_vector[i_val[1],atomindex] += g_vals[1]
                    temp_vector[1,j_index] += g_vals[2]
                    temp_vector[1,k_index] += g_vals[3]
                end
            end
        end
    end

    #create rows 2,3 where j=Cu,k=Zn
    
        for j_index in 1:n1
            if j_index !=atomindex
                for k_index in 1:n2
                    if k_index != atomindex
                        g_vals = new_angular_symm_vals(newposition,positions[atomindex],positions[j_index],positions[k_index],new_dist2_vector[indexj],new_dist2_vector[indexk],dist2_mat[indexi,indexj],dist2_mat[indexi,indexk],dist2_mat[indexj,indexk],new_f_vector[indexj],new_f_vector[indexk],f_matrix[indexi,indexj],f_matrix[indexi,indexk],f_matrix[indexj,indexk],η,λ,ζ,tpz)

                        temp_vector[ival[2],atomindex] += g_vals[1]
                        temp_vector[2,j_index] += g_vals[2]
                        temp_vector[3,k_index] += g_vals[3]
                    end
                end
            end
        end

    for j_index in n1+1:N
        if j_index != atomindex
            for k_index in j_index+1:N
                if k_index != atomindex
                    g_vals = new_angular_symm_vals(newposition,positions[atomindex],positions[j_index],positions[k_index],new_dist2_vector[indexj],new_dist2_vector[indexk],dist2_mat[indexi,indexj],dist2_mat[indexi,indexk],dist2_mat[indexj,indexk],new_f_vector[indexj],new_f_vector[indexk],f_matrix[indexi,indexj],f_matrix[indexi,indexk],f_matrix[indexj,indexk],η,λ,ζ,tpz)

                    temp_vector[i_val[3],atomindex] += g_vals[1]
                    temp_vector[4,j_index] += g_vals[2]
                    temp_vector[4,k_index] += g_vals[3]
                end
            end
        end
    end

    if atomindex > n1
        g_vector[1,:] = temp_vector[4,:]
        g_vector[2,:] = temp_vector[1,:] .+ temp_vector[3,:]
        g_vector[3,:] = temp_vector[2,:]
    else
        g_vector[1,:] = temp_vector[1,:]
        g_vector[2,:] = temp_vector[2,:] .+ temp_vector[4,:]
        g_vector[3,:] = temp_vector[3,:]
    end

    return g_vector
end

calc_new_symmetry_value! (generic function with 2 methods)

In [16]:
function testcalc_new_symmetry_value!(g_vector,positions,newposition,atomindex,dist2_mat,new_dist2_vector,f_matrix,new_f_vector,n1,n2,symmetry_function::AngularType3)

    N = n1+n2 

    η,λ,ζ,tpz = symmetry_function.eta,symmetry_function.lambda,symmetry_function.zeta,symmetry_function.tpz

    # temp_vector = zeros(4,N)
    idx = ifelse(atomindex > n1, [3,1] , [1,3])
    #calc i-CuCu -> row 1 
    #if i=Cu row1=111, if i=Zn row1=110
    #if Cu i -> row 1 if Zn i -> row 3
    for j_index in 1:n1
        if j_index != atomindex
            for k_index in j_index+1:n1
                if k_index != atomindex

                    g_vals = new_angular_symm_vals(newposition,positions[atomindex],positions[j_index],positions[k_index],new_dist2_vector[indexj],new_dist2_vector[indexk],dist2_mat[indexi,indexj],dist2_mat[indexi,indexk],dist2_mat[indexj,indexk],new_f_vector[indexj],new_f_vector[indexk],f_matrix[indexi,indexj],f_matrix[indexi,indexk],f_matrix[indexj,indexk],η,λ,ζ,tpz)

                    g_vector[idx[1],atomindex] += g_vals[1]
                    g_vector[1,j_index] += g_vals[2]
                    g_vector[1,k_index] += g_vals[3]
                end
            end
        end
    end
        # calculate i-ZnZn -> row 2
        # If Zn then row2=111, if Cu then row2=110
        #if Cu i -> row 3 if Zn i -> row 2
    for j_index in n1+1:N
        if j_index != atomindex
            for k_index in j_index+1:N
                if k_index != atomindex
                    g_vals = new_angular_symm_vals(newposition,positions[atomindex],positions[j_index],positions[k_index],new_dist2_vector[indexj],new_dist2_vector[indexk],dist2_mat[indexi,indexj],dist2_mat[indexi,indexk],dist2_mat[indexj,indexk],new_f_vector[indexj],new_f_vector[indexk],f_matrix[indexi,indexj],f_matrix[indexi,indexk],f_matrix[indexj,indexk],η,λ,ζ,tpz)
    
                    g_vector[idx[2],atomindex] += g_vals[1]
                    g_vector[2,j_index] += g_vals[2]
                    g_vector[2,k_index] += g_vals[3]
                end
            end
        end
    end
#calc i-CuZn
    if atomindex > n1 
        # i = Zn => ZnCuZn, row1=110,row2=111,row3=100
        for j_index in 1:n1
            if j_index !=atomindex
                for k_index in 1:n2
                    if k_index != atomindex
                        g_vals = new_angular_symm_vals(newposition,positions[atomindex],positions[j_index],positions[k_index],new_dist2_vector[indexj],new_dist2_vector[indexk],dist2_mat[indexi,indexj],dist2_mat[indexi,indexk],dist2_mat[indexj,indexk],new_f_vector[indexj],new_f_vector[indexk],f_matrix[indexi,indexj],f_matrix[indexi,indexk],f_matrix[indexj,indexk],η,λ,ζ,tpz)

                        g_vector[1,atomindex] += g_vals[1]
                        g_vector[3,j_index] += g_vals[2]
                        g_vector[1,k_index] += g_vals[3]
                    end
                end
            end
        end
    #if Cu then i,j then row 1=111, row2=110 => i,j->2 k->3
    else
        for j_index in 1:n1
            if j_index != atomindex
                for k_index in 1:n2
                    if k_index != atomindex
                        g_vals = new_angular_symm_vals(newposition,positions[atomindex],positions[j_index],positions[k_index],new_dist2_vector[indexj],new_dist2_vector[indexk],dist2_mat[indexi,indexj],dist2_mat[indexi,indexk],dist2_mat[indexj,indexk],new_f_vector[indexj],new_f_vector[indexk],f_matrix[indexi,indexj],f_matrix[indexi,indexk],f_matrix[indexj,indexk],η,λ,ζ,tpz)
                        g_vector[2,atomindex] += g_vals[1]
                        g_vector[2,j_index] += g_vals[2]
                        g_vector[3,k_index] += g_vals[3]
                    end
                end
            end
        end
    end
    #If copper the rows are (111,110,100), If Zn they are (110,111,100)

    if atomindex > n1
        g_vector[1:2,:] = g_vector[2:-1:1,:]
    end

    return g_vector
end

testcalc_new_symmetry_value! (generic function with 1 method)

In [17]:
function total_symm_calc(g_mat,positions,newposition,atomindex,dist2_mat,new_dist2_vector,f_mat,new_f_vector,radsymmfunctions,angsymmfunctions,nrad,nang,n1,n2)
    
    g_mat = zeros(nrad*2 + nang*3 , length(positions))

    for g_index in 1:nrad
        idx=(g_index-1)*2+1

        g_mat[idx:idx+1,:] = calc_new_symmetry_value!(g_mat[idx:idx+1,:],atomindex,dist2_mat,new_dist2_vector,f_matrix,new_f_vector,n1,n2,radsymmfunctions[g_index])
 
    end
    for g_index in 1:nang 
        idx = nrad*2 + (g_index-1)*3 + 1 

        g_mat[idx:idx+2,:] = calc_new_symmetry_value!(g_mat[idx:idx+2,:],positions,newposiiton,atomindex,dist2_mat,new_dist2_vector,f_matrix,new_f_vector,n1,n2,angsymmfunctions[g_index])
    end

    return g_mat
end
function testtotal_symm_calc(g_mat,positions,newposition,atomindex,dist2_mat,new_dist2_vector,f_mat,new_f_vector,radsymmfunctions,angsymmfunctions,nrad,nang,n1,n2)
    
    g_mat = zeros(nrad*2 + nang*3 , length(positions))

    for g_index in 1:nrad
        idx=(g_index-1)*2+1

        g_mat[idx:idx+1,:] = calc_new_symmetry_value!(g_mat[idx:idx+1,:],atomindex,dist2_mat,new_dist2_vector,f_matrix,new_f_vector,n1,n2,radsymmfunctions[g_index])
 
    end
    for g_index in 1:nang 
        idx = nrad*2 + (g_index-1)*3 + 1 

        g_mat[idx:idx+2,:] = testcalc_new_symmetry_value!(g_mat[idx:idx+2,:],positions,newposiiton,atomindex,dist2_mat,new_dist2_vector,f_matrix,new_f_vector,n1,n2,angsymmfunctions[g_index])
    end

    return g_mat
end

testtotal_symm_calc (generic function with 1 method)

We have written the essential functions we aim to use. On to initialising states and doing some testing.

In [34]:
gmat

88×38 Matrix{Float64}:
 0.369506     0.281048   0.359579     0.329095     …  0.71955     0.491495
 0.010325     0.141364   0.0250313    0.0701882       0.12057     0.106197
 0.41575      0.301073   0.409381     0.391113        0.766125    0.519433
 0.00325322   0.163532   0.0121555    0.0376877       0.144834    0.134867
 0.434908     0.307028   0.431018     0.420083        0.765621    0.520055
 0.00124181   0.175765   0.006551     0.0214746    …  0.161052    0.15313
 0.418896     0.282958   0.418639     0.417932        0.664259    0.461725
 1.71073e-5   0.200526   0.000397098  0.00144         0.204062    0.194975
 0.0730804    0.0489541  0.0730804    0.0730804       0.113693    0.0819406
 8.38881e-15  0.0386131  1.53215e-10  5.24035e-10     0.0534105   0.045916
 ⋮                                                 ⋱              
 0.149938     0.072194   0.149911     0.149812        0.0         0.0
 0.0          0.0        0.0          0.0          …  0.279725    0.203601
 2.21847e-11  0

In [19]:
import ParallelTemperingMonteCarlo.EnergyEvaluation.set_variables
function set_variables(config::Config{N,BC,T},dist2_mat,pot::RuNNerPotential2Atom{nrad,nang}) where {N,BC,T} where {nrad,nang}
    f_matrix = cutoff_function.(sqrt.(dist2_mat),Ref(pot.r_cut))
    g_matrix = total_symm_calc(config.pos,dist2_mat,f_matrix,pot.radsymfunctions,pot.angsymfunctions,nrad,nang,30,8)
    return NNPVariables{T}(zeros(N) , zeros(N) , g_matrix , f_matrix ,zeros(nrad*2 + nang*3 , N) , zeros(N))
end

set_variables (generic function with 5 methods)

In [30]:
file = open("input.data","w+")
write(file,"begin\n")
for pos in start_config.pos
    write(file,"atom $(pos[1]) $(pos[2]) $(pos[3]) Cu 0.0 0.0 0.0 0.0 0.0 \n")
end
write(file,"energy 0.000\n")
write(file,"charge 0.000\n")
write(file,"end")
close(file)


In [152]:
envec = zeros(38)

#envec = forward_pass(gmat,38,runnerpotential.nnp1)

envec[1:32] = forward_pass(gmat[:,1:32],32,runnerpotential.nnp1)
envec[33:38] = forward_pass(gmat[:,33:38],6,runnerpotential.nnp2)
envec

38-element Vector{Float64}:
 -0.08970620224975694
 -0.0932971052765097
 -0.08936465260792263
 -0.08933115691641783
 -0.0892651606290647
 -0.12310021962793745
 -0.08984288352159023
 -0.09025549786766054
 -0.09096621591762226
 -0.08967929905177578
  ⋮
 -0.08966131154602258
 -0.1621760035437439
 -0.08870780240338014
 -0.009906940633193058
 -0.0101628475988158
 -0.01045000575102868
 -0.008848849712083258
 -0.056601384907602144
 -0.023218867456933637

In [153]:
sum(envec)


-3.616458655936369

In [65]:
function get_gnorm_etc(gnormvec)
    g_norm = 1/(gnormvec[1] - gnormvec[2])
    g_offset = -1* gnormvec[2] * g_norm
    return g_norm , g_offset
end

get_gnorm_etc (generic function with 1 method)

In [51]:
for radsym in runnerpotential.radsymfunctions
    println(radsym.G_offset)
end

[-0.029892621990594427, -0.0, -0.0, -0.0]
[-0.013430592462258787, -0.0, -0.0, -0.0]
[-0.0067056953127401205, -0.0, -0.0, -0.0]
[-0.00026435144512647774, -0.0, -0.0, -0.0]
[-0.0, -0.0, -0.0, -0.0]


In [69]:
cunorms = zeros(2,88)
znorms = zeros(2,88)
for i in 1:88
    cunorms[1,i],cunorms[2,i] = get_gnorm_etc(G_value_vec[i])
    znorms[1,i],znorms[2,i] = get_gnorm_etc(G_value_vec_b[i])
end


In [107]:
xx = readdlm("symfunctions.out")
xx = xx[2:end,2:end]

38×88 Matrix{Any}:
 6.02005  0.105055  3.22172  0.0174727  2.10943  …  1.7364e-6    0.0
 4.68674  1.43837   2.36088  0.878314   1.49859     0.00288527   0.000893812
 5.87041  0.254691  3.17391  0.0652858  2.09085     4.1845e-5    2.0e-10
 5.41095  0.714158  3.03678  0.202417   2.03862     0.000147004  3.024e-7
 5.58551  0.539592  3.03299  0.20621    2.01397     0.000515291  1.5133e-6
 7.0475   1.11858   3.86335  0.587214   2.56036  …  0.00287141   0.000110013
 5.77065  0.354459  3.13697  0.102225   2.07547     7.54229e-5   1.659e-7
 4.24724  1.87787   1.99345  1.24575    1.21279     0.00388641   0.00192228
 4.77326  1.35185   2.57146  0.667738   1.70036     0.00223599   0.000155539
 5.41292  0.712189  2.98063  0.258561   1.99417     0.00061629   3.676e-6
 ⋮                                               ⋱               
 5.87966  0.245442  3.17521  0.0639879  2.0912      3.80321e-5   1.23e-8
 9.1199   2.37241   4.86086  1.3566     3.151    …  0.00808112   0.000839822
 5.35299  0.772112 

In [108]:
for index in 1:32
    xx[index,:] .*= cunorms[1,:]
    xx[index,:] .+=cunorms[2,:]
end

In [109]:
for index in 33:38
    xx[index,:] .*=znorms[1,:]
    xx[index,:] .+= znorms[2,:]
end

In [150]:
v = xx - transpose(gmat)
for i in 1:38
    for j in 1:88
        if abs(v[i,j]) < 1e-8
            v[i,j] =0.
        end
    end
end

In [151]:
v[:,11:end]

38×78 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …   0.0         0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0      1.52695e-8  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0      0.0         0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     -3.06289e-8  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     -5.08801e-8  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …   0.0         0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     -1.30796e-8  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0      3.02577e-8  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     -5.22528e-8  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0      3.18581e-8  0.0  0.0  0.0
 ⋮                        ⋮                   ⋱               ⋮         
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0      0.0         0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  -1.60663e-8  0.0  0

In [132]:
xx[:,11:end]

38×78 Matrix{Any}:
 0.242938  0.0174495  0.00010192   …  0.204445   6.65478e-5  0.0
 0.154095  0.199215   0.0304859       0.108954   0.110579    0.0652122
 0.231913  0.0438742  0.000160359     0.203431   0.00160372  1.45919e-8
 0.195755  0.125329   0.00535249      0.200765   0.00563396  2.2063e-5
 0.20972   0.0927801  0.00439159      0.191424   0.0197486   0.00011041
 0.283732  0.200572   0.0145547    …  0.264738   0.110047    0.00802654
 0.221959  0.0655069  0.00234836      0.202578   0.0028906   1.2104e-5
 0.114557  0.270799   0.0529366       0.0576465  0.148947    0.140249
 0.152054  0.214804   0.0202332       0.144031   0.0856946   0.0113481
 0.19146   0.130318   0.0104639       0.188816   0.0236194   0.0002682
 ⋮                                 ⋱  ⋮                      
 0.229739  0.0482818  0.00094308      0.203527   0.00145759  8.97404e-7
 0.378883  0.510018   0.0510873    …  0.301691   0.30971     0.0612731
 0.190613  0.136183   0.00678915      0.190463   0.0212053   0.0001117

In [149]:
transpose(gmat)[:,11:end]

38×78 Matrix{Float64}:
 0.242938  0.0174495  0.00010192   …  0.204445   6.6548e-5   2.42411e-9
 0.154095  0.199215   0.0304859       0.108954   0.110579    0.0652122
 0.231913  0.0438742  0.000160359     0.203431   0.00160372  1.80857e-8
 0.195755  0.125329   0.00535249      0.200765   0.00563396  2.206e-5
 0.20972   0.0927801  0.00439159      0.191424   0.0197486   0.00011041
 0.283732  0.200572   0.0145547    …  0.264738   0.110047    0.00802654
 0.221959  0.0655069  0.00234836      0.202578   0.00289059  1.21029e-5
 0.114557  0.270799   0.0529366       0.0576465  0.148947    0.140249
 0.152054  0.214804   0.0202332       0.144031   0.0856946   0.0113481
 0.19146   0.130318   0.0104639       0.188816   0.0236194   0.000268198
 ⋮                                 ⋱  ⋮                      
 0.229739  0.0482818  0.00094308      0.203527   0.00145759  8.98311e-7
 0.378883  0.510018   0.0510873    …  0.301691   0.30971     0.0612731
 0.190613  0.136183   0.00678915      0.190463   0.021205